### Thread class로 thread 생성
threading.Thread
  #### 파라미터 : 
      name 은 쓰레드 명을 정의하지만 없으면 자동으로 생성됨
      target은 처리 함수
      args는 함수에 들어갈 인수전달
  #### 메소드 : 
     thread.start() : thread를 start 메소드로 실행시켜야 함
     thread.join() : thread를 종료
     getName(), setName() : thread name 조회 및 세팅

In [ ]:
%%writefile work.py
import threading

class Worker(threading.Thread):
    # Our workers constructor, note the super() method which is vital if we want this
    # to function properly
    def __init__(self, num):
        super(Worker, self).__init__()
        self.num = num

    def run(self):
        print(" worker num %s " % (self.num))
        for i in range(10):
            print(i)
        

In [ ]:
%%writefile work_thread.py
import threading 
from work import Worker

def main():
    # This initializes ''thread1'' as an instance of our Worker Thread
    thread1 = Worker(1)
    thread2 = Worker(2)
    thread3 = Worker(3)
    # This is the code needed to run our newly created thread
    thread1.start()
    thread2.start()
    thread3.start()

if __name__ == "__main__":  
    main()

In [ ]:
!python work_thread.py

In [ ]:
import threading

def worker():
    """thread worker function"""
    print('Worker')
    return

threads = []
for i in range(3):
    t = threading.Thread(target=worker)
    threads.append(t)
    t.start()
    print("thread name ",t.name)   
    print("thread .ident", t.ident)
    print("thread daemon ", t.daemon)
    

for th in threads:
    th.join()

In [ ]:
import threading

def worker():
    """thread worker function"""
    print('Worker')
    return

threads = []
for i in range(5):
    t = threading.Thread(target=worker)
    threads.append(t)
    t.start()
print("threading count ",threading.active_count())   
print(threads)
print(threads[0].__dict__)

for th in threads:
    th.join()

In [ ]:
import threading

def worker(num):
    """thread worker function"""
    print('Worker: %s' % num)
    return

threads = []
for i in range(5):
    t = threading.Thread(target=worker, args=(i,))
    threads.append(t)
    t.start()
    
print("threading count ",threading.active_count())   

for t in threads :
    t.join()
    


### 현재 thread 조회하기

threading.main_thread() : 실행하는 main thread를 조회

threading.currentThread() : 현재 실행되는 thread 객체 가져오기

예시 threading.currentThread().getName() :  현재 thread 객체의 name을 읽어옴

threading.active_count() : 현재 실행되는 thread 갯수

threading.enumerate() : 현재 실행되는 thread 전체 갯수에 대한 정보를 출력

In [ ]:
import threading
import time

def worker():
    print(threading.currentThread().getName(), 'Starting')
    time.sleep(2)
    print(threading.currentThread().getName(), 'Exiting')

def my_service():
    print(threading.currentThread().getName(), 'Starting')
    time.sleep(3)
    print(threading.currentThread().getName(), 'Exiting')

t = threading.Thread(name='my_service', target=my_service)
w = threading.Thread(name='worker', target=worker)
w2 = threading.Thread(target=worker) # use default name

w.start()
w2.start()
t.start()

print("threading count ",threading.active_count())  
w.join()
w2.join()
t.join()


In [ ]:
import threading

print("threading count ",threading.active_count()) 

print("Main thread ", threading.main_thread())

# 현행 하나의 thread만 가져옴
print(threading.currentThread())


In [ ]:
import threading

print("threading count ",threading.active_count()) 

print("Main thread ", threading.main_thread())



# 프로세스 내의 모든 thread만 가져옴
a = threading.enumerate()
for i in a :
    print(i)

In [ ]:
import threading

print("threading count ",threading.active_count()) 

print("Main thread ", threading.main_thread())


In [ ]:
from time import sleep
from random import random
from threading import Thread, local

data = local()

def bar():
   
    print("I'm called from", data.v)


class T(Thread):
    
    def run(self):
        
        sleep(random())
        data.v = self.getName()   # Thread-1 and Thread-2 accordingly
        sleep(1)
        bar()
        
T().start()


## 동기화 처리

threading.Condition


   ### 메소드 
    
    wait 메소드 : 통지 할 때까지 또는 시간 초과가 발생할 때까지 기다립니다.
                이 메서드를 호출 할 때 호출 스레드가 잠금을 획득하지 않은 경우, 런타임 오류가 발생합니다.
                
                
    notifyAll 메소드 : 대기 중의 모든 스레드에를 캐우며,  
                   이 메서드를 호출 할 때 호출 스레드가 잠금을 획득하지 않은 경우 RuntimeError에가 발생합니다.

In [ ]:
condition = threading.Condition()
print(condition)
print(dir(condition))

In [ ]:
import logging
import threading
import time

logging.basicConfig(level=logging.DEBUG,
                    format='%(asctime)s (%(threadName)-2s) %(message)s',
                    )

def consumer(cond):
    """wait for the condition and use the resource"""
    logging.debug('Starting consumer thread')
    t = threading.currentThread()
    print(" consumer ", t)
    with cond:
        cond.wait()
        logging.debug('Resource is available to consumer')

def producer(cond):
    """set up the resource to be used by the consumer"""
    
    t = threading.currentThread()
    logging.debug('Starting producer thread')
    print(" producer ", t)
    with cond:
        logging.debug('Making resource available')
        cond.notifyAll()

condition = threading.Condition()
c1 = threading.Thread(name='c1', target=consumer, args=(condition,))
c2 = threading.Thread(name='c2', target=consumer, args=(condition,))
p = threading.Thread(name='p', target=producer, args=(condition,))


c1.start()
time.sleep(2)
c2.start()
time.sleep(2)
p.start()

In [ ]:
import threading, time

def myThread(id):
    for i in range(10):
        print( 'id=%s --> %s' % (id, i))
        # CPU 양보
        time.sleep(0)

# thread 객체를 모아둘 리스트
threads = []

for i in range(2):
    # myThread를 실행하는 thread 생성
    th = threading.Thread(target=myThread, args=(i,))
    # thread 시작
    th.start()
    # thread 객체리스트에 추가
    threads.append(th)
    
print(threads)
# thread 종료까지 대기
for th in threads:
    th.join()

print ('Exiting')

###  subclass  thread

    사용자 정의 클래스에서 threading.Thread 클래스를 상속받고 처리
    
      
   #### 오버라이딩 

        thread.run : 메소드를 사용자 정의 클래스에 오버라이딩
 
             인자를 생성하고 start 메소드를 실행하면 run 메소드가 작동됨

In [ ]:
import threading


class MyThread(threading.Thread):

    def run(self):
        print('running', self.name)
        return

for i in range(5):
    t = MyThread()
    t.start()

In [ ]:
import sys
import threading

class DestinationThread(threading.Thread):

    def run(self):
        if sys.version_info[0] == 2:
            print("version 2")
            self._Thread__target(*self._Thread__args, **self._Thread__kwargs)
        else: # assuming v3
            print("version 3")
            self._target(*self._args, **self._kwargs)

def func(a, k):
    print("func(): a=%s, k=%s" % (a, k))

thread = DestinationThread(target=func, args=(1,), kwargs={"k": 2})
for i in thread.__dict__.keys() :
    print(i)
    
thread.start()
thread.join()

In [ ]:
import threading

destination_name = 'name'
destination_config = "config"

class Destination:
    def run(self, name, config):
        print('In thread', name, config)

destination = Destination()
thread = threading.Thread(target=destination.run,
    args=(destination_name, destination_config))
print(thread.__dict__)

thread.start()

In [ ]:
thread.__dict__

### Thread 를 함수(_start_new_threadthread )로 생성

 thread._start_new_thread(func, args, kwargs=None)
     func = thread 실행 함수
     args = func에 넘겨줄 인수
     kwargs = 키워드 인수

thread는 자동 실행 됨

In [ ]:

import threading, time

result_values= []

# thread에서 실행될 함수
def counter(id): 
    for i in range(5):
        print('id %s --> %s' % (id, i))
        
        result_values.append((id,i))
        time.sleep(0.1)

# thread 5개 실행
for i in range(5):
    threading._start_new_thread(counter, (i,))

# thread가 다 돌 때까지 대기
time.sleep(1)
print(result_values)
print( 'Exiting')

In [ ]:
dir(threading)

In [ ]:
help(threading.currentThread)

## 프로세스 처리

'authkey',
 'daemon',
 'exitcode',
 'ident',
 'is_alive',
 'join',
 'name',
 'pid',
 'run',
 'sentinel',
 'start',
 'terminate'

In [ ]:
%%writefile mp_sample.py
import multiprocessing as mp
import os
import time

def func(x) :
    
    print(" process id ", os.getpid())
    print(" parents process id",os.getppid())
    
def main() :      
    for i in range(3) :
        p = mp.Process(target=func, args=(i,))
        print(p)
        p.start()
    time.sleep(2)
    p.join()
    
if __name__ == "__main__" :
    main()

In [ ]:
!python mp_sample.py

In [ ]:
import multiprocessing as mp
dir(mp.Process)

In [ ]:
%%writefile mp_sample1.py
import multiprocessing as mp
import os
import time

def func(x) :
    print("func processing")
    
def main() :      
    for i in range(1) :
        p = mp.Process(target=func, args=(i,))
        p.start()
       
        print("process authkey ",p.authkey)
        print("process daemon ",p.daemon) # Return whether process is a daemon
        print("process name ",p.name)
        print("process ident ",p.ident) #Return identifier (PID) of process or `None` if it has yet to start
        print("process pid ",p.pid)
        # Return a file descriptor (Unix) or handle (Windows) suitable for waiting for process termination.
        print("process sentinel ",p.sentinel)
        
    time.sleep(2)
    p.join()
    
if __name__ == "__main__" :
    main()
    

In [ ]:
!python mp_sample1.py

In [ ]:
%%writefile import_worker.py
def worker():
    """worker function"""
    print('Worker')
    return

In [ ]:
%%writefile process_main.py
import multiprocessing
import import_worker

if __name__ == '__main__':
    jobs = []
    for i in range(5):
        p = multiprocessing.Process(target=import_worker.worker)
        jobs.append(p)
        p.start()

In [ ]:
!python process_main.py

In [ ]:
%%writefile process_variance.py
import multiprocessing
import time

def worker():
    name = multiprocessing.current_process().name
    print(name, 'Starting')
    time.sleep(2)
    print( name, 'Exiting')

def my_service():
    name = multiprocessing.current_process().name
    print(name, 'Starting')
    time.sleep(3)
    print( name, 'Exiting')

if __name__ == '__main__':
    service = multiprocessing.Process(name='my_service', target=my_service)
    worker_1 = multiprocessing.Process(name='worker 1', target=worker)
    worker_2 = multiprocessing.Process(target=worker) # use default name

    worker_1.start()
    worker_2.start()
    service.start()

In [ ]:
!python process_variance.py

In [ ]:
%%writefile process_daemon.py
import multiprocessing
import time, sys

def daemon():
    p = multiprocessing.current_process()
    print('Starting:', p.name, p.pid)
    sys.stdout.flush() # 정의하지 않으면 출력이 안됨
    time.sleep(2)
    print('Exiting :', p.name, p.pid)
    sys.stdout.flush()

def non_daemon():
    p = multiprocessing.current_process()
    print('Starting:', p.name, p.pid)
    print('Exiting :', p.name, p.pid)

if __name__ == '__main__':
    d = multiprocessing.Process(name='daemon', target=daemon)
    d.daemon = True

    n = multiprocessing.Process(name='non-daemon', target=non_daemon)
    n.daemon = False

    d.start()
    time.sleep(1)
    n.start()
    print("daemon processing status ",d.is_alive())

In [ ]:
!python process_daemon.py

In [ ]:
%%writefile process_terminate.py
import multiprocessing
import time

def slow_worker():
    print('Starting worker')
    time.sleep(0.1)
    print('Finished worker')

if __name__ == '__main__':
    p = multiprocessing.Process(target=slow_worker)
    print('BEFORE:', p, p.is_alive())
    
    p.start()
    print('DURING:', p, p.is_alive())
    
    p.terminate()
    print('TERMINATED:', p, p.is_alive())

    p.join()
    print('JOINED:', p, p.is_alive())

In [ ]:
!python process_terminate.py

In [ ]:
%%writefile process_daemon1.py
import multiprocessing
import time, sys, os, signal

def daemon():
    p = multiprocessing.current_process()
    print('Starting:', p.name, p.pid)
    sys.stdout.flush() # 정의하지 않으면 출력이 안됨
    time.sleep(2)
    print('Exiting :', p.name, p.pid)
    sys.stdout.flush()

if __name__ == '__main__':
    d = multiprocessing.Process(name='daemon', target=daemon)
    d.daemon = True

    d.start()
    time.sleep(1)
    print('DURING:', d, d.is_alive())
    
    d.terminate()   # daemon은 죽지 않음
    print('TERMINATED:', d, d.is_alive())
    
    os.kill(d.pid, signal.CTRL_C_EVENT)
    print('Kill :', d, d.is_alive())
    
    sys.exit() # daemon은 현재 빠져나가야 함
    print('sys exit :', d, d.is_alive())

In [ ]:
!python process_daemon1.py

In [ ]:
import os, signal
help(signal)

In [ ]:
%%writefile process_pool.py
import multiprocessing

def funSquare(num):
    return num ** 2

if __name__ == '__main__':
    pool = multiprocessing.Pool()
    print(pool)
    results = pool.map(funSquare, range(10))
    print(results)

In [ ]:
!python process_pool.py

In [ ]:
import multiprocessing
help(multiprocessing.Process)

In [ ]:
%%writefile process_test1.py
from multiprocessing import Process

def say_hello(name='world'):
    print("Hello, %s" % name)
    
if __name__ == "__main__" :
    p = Process(target=say_hello, name="say_hello")
   
    p.start()
    print(p.name)
    print(p.pid)
    p.join()

In [ ]:
!python process_test1.py

In [ ]:
%%writefile process_test2.py
from multiprocessing import  Process, Queue

q = Queue()

def put_queue(q,msg):
    q.put(msg)
    print("put , %s" % q._buffer)
    
def get_queue(q):
    msg = q.get()
    print("queue buffer , %s" % q._buffer)
    print("get, %s" % msg)
    
if __name__ == "__main__" :
    msg = "Why hello there!"
    p = Process(target=put_queue, name="put_queue", args=(q,msg))
    g = Process(target=get_queue, name="get_queue", args=(q,))
    
    p.start()
    g.start()
    p.join()
    g.join()


In [ ]:
!python process_test2.py

In [ ]:
%%writefile process_test3.py
from multiprocessing import Process, Queue, current_process
import random

process_str = []
def rand_num(queue):
    num = random.random()
    p = current_process()
    queue.put([p.name,num])
    process_str.append(p)
    print(p, num)
    
    
if __name__ == "__main__":
    queue = Queue()
    processes = [Process(target=rand_num, args=(queue,)) for x in range(4)]
    
    for p in processes:
        p.start()
      
    for p in processes:
        print(queue.get()) 
        

    for p in processes:
        p.join()


In [ ]:
!python process_test3.py

In [ ]:
!python process_test3.py

In [ ]:
import collections

a = collections.deque([(1,1),(2,2)])
print(a)

from multiprocessing import Process, Queue, current_process

help(Queue().put)
a = Queue()
a.put((1,1))
print(a._buffer)
print(a.get())


In [ ]:
from multiprocessing import Lock

l = Lock()

l.acquire()
print('Ha! Only I can write to stdout!')
l.release()

In [ ]:
%%writefile process_test.py
import os
 
from multiprocessing import Process
 
def doubler(number):
    """
    A doubling function that can be used by a process
    """
    result = number * 2
    proc = os.getpid()
    print('{0} doubled to {1} by process id: {2}'.format(
        number, result, proc))
    
    
if __name__ == '__main__':
    numbers = [5, 10, 15, 20, 25]
    procs = []
 
    for index, number in enumerate(numbers):
        proc = Process(target=doubler, args=(number,))
        procs.append(proc)
        proc.start()
 
    for proc in procs:
        proc.join()

In [ ]:
!python process_test.py

In [ ]:
from multiprocessing import Process
help(Process)

In [ ]:
%%writefile process_test1.py

 
from multiprocessing import Process, current_process
 
def doubler(number):
    
    print('number{0} process id: {1}'.format(
        number, current_process()))
    
    
if __name__ == '__main__':
    numbers = [5, 10, 15, 20, 25]
    procs = []
 
    for index, number in enumerate(numbers):
        proc = Process(target=doubler, args=(number,))
        procs.append(proc)
        proc.start()
 
    for proc in procs:
        proc.join()

In [ ]:
!python process_test1.py

In [ ]:
%%writefile process_test3.py
import multiprocessing
import time

def worker(*args,**kwargs):
    proc = multiprocessing.current_process()
    print(proc, args,kwargs)
    time.sleep(2)
    print(proc.__dict__ )

if __name__ == '__main__':
    
    worker_1 = multiprocessing.Process(name='worker 1', 
                                       target=worker, 
                                       args=(1,2,3),
                                       kwargs={'a':1,'b':2})
    
    worker_1.start()
  
    worker_1.join(4)

In [ ]:
!python process_test3.py

In [ ]:
%%writefile process_test4.py
import multiprocessing
import time

data = ['a','b','c','d','e','f','g','h']
data1 = [1,2,3,4,5,6,7,8]
   

def mp_worker(*args):
    print(args)
    inputs = args[0][0]
    the_time = args[0][1]
    print(" Processs %s\tWaiting %s seconds" % (inputs, the_time))
    time.sleep(int(the_time))
    print(" Process %s\tDONE" % inputs)

def mp_handler():
    subdata = zip(data, data1)
    print(list(subdata))
    
    for i in range(8) :
        p = multiprocessing.Pool(2)
        p.map(mp_worker, list(subdata))
    p.close()

if __name__ == '__main__':
    mp_handler()

In [ ]:
!python process_test4.py

In [ ]:
import multiprocessing
a = multiprocessing.Pool()
help(a.close)

In [ ]:
%%writefile process_test5.py
import multiprocessing
import time

data = (
    ['a', '2'], ['b', '4'], ['c', '6'], ['d', '8'],
    ['e', '1'], ['f', '3'], ['g', '5'], ['h', '7']
)

def mp_worker(*args):
    inputs = args[0][0]
    the_time = args[0][1]
    print(" Processs %s\tWaiting %s seconds" % (inputs, the_time))
    time.sleep(int(the_time))
    print(" Process %s\tDONE" % inputs)

def mp_handler():
    p = multiprocessing.Pool(2)
    p.map(mp_worker, data)

if __name__ == '__main__':
    mp_handler()

In [ ]:
!python process_test5.py

In [ ]:
%%writefile process_test6.py
import multiprocessing

def myfunction(i,event):
    print(args)
    i = 20
    event = args[1]
    if not event.is_set():
        print(i) 
    if i == 20:
        event.set()

if __name__ == "__main__":
    pool= multiprocessing.Pool(2) 
    m = multiprocessing.Manager()
    event = m.Event()
    for i in range(100):
        pool.apply_async(myfunction , (i, event))
    pool.close()

    event.wait()  # We'll block here until a worker calls `event.set()`
    pool.terminate() # Terminate all processes in the Pool

In [ ]:
!python process_test6.py

In [ ]:
%%writefile process_test7.py

import multiprocessing
import time

def myfunction(i):
    
    print(i, " excute process ",multiprocessing.current_process())
    time.sleep(0.5)
    return i*i

if __name__ == "__main__":
    print("__name__")
    p = multiprocessing.Pool(2) 
    result = p.map(myfunction, range(10))
    print(" result ", result)
   

In [ ]:
!python process_test7.py

In [ ]:
%%writefile process_test8.py

import multiprocessing
def myfunction(*args):
    i = 0
    print(args)
    for i in args :
        i += 1
        
    print(i)
    if (i == 20):
        sys.exit()

if __name__ == "__main__":
    print("__name__")
    p = multiprocessing.Pool(2) 
    a = p.apply_async(myfunction, args=(1,2,3,4)).get()
    print(a)
   

In [ ]:
!python process_test8.py

In [ ]:
%%writefile process_test9.py

import multiprocessing as mp
import time

def foo_pool(x):
    time.sleep(2)
    return x*x

result_list = []
def log_result(result):
    # This is called whenever foo_pool(i) returns a result.
    # result_list is modified only by the main process, not the pool workers.
    result_list.append(result)

def apply_async_with_callback():
    pool = mp.Pool()
    for i in range(10):
        pool.apply_async(foo_pool, args = (i, ), callback = log_result)
    pool.close()
    pool.join()
    print(result_list)

if __name__ == '__main__':
    apply_async_with_callback()

In [ ]:
!python process_test9.py

In [ ]:
import multiprocessing as mp

print(type(mp.Pool()))

print(type(mp.Pool().apply_async))
print(help(mp.Pool().apply_async))

In [ ]:
print(help(mp.Pool().close))

In [ ]:
print(help(mp.Pool().join))

In [ ]:
%%writefile process_apply.py
import multiprocessing
import time
 
def square(x):
    # This is is reeeeally slow way to square numbers.
    print(" excute process ",multiprocessing.current_process())
    time.sleep(0.5)
    return x**2
 
if __name__ == "__main__" : 
    print("Creating pool with 3 workers")
    pool = multiprocessing.Pool(processes=3)
    print(pool._pool)
    print("Invoking apply(square, 3)")
    print("Result: %s" % (pool.apply(square, [3]),))
    print("Result: %s" % (pool.apply(square, [9]),))
    pool.close()
    pool.join()


In [ ]:
!python process_apply.py

In [ ]:
%%writefile process_applyasync.py
import multiprocessing
import time
 
def square(x):
    # This is is reeeeally slow way to square numbers.
    print(" excute process ",multiprocessing.current_process())
    time.sleep(0.5)
    return x**2
 
if __name__ == "__main__" : 
    print("Creating pool with 3 workers")
    pool = multiprocessing.Pool(processes=3)
    print("Invoking apply_async(square, 4)")
    result = pool.apply_async(square, [4])
    print("Waiting for result...")
    start_time = time.time()
    print("Result: %s (%.2f secs)" % (result.get(), time.time() - start_time))
    pool.close()
    pool.join()


In [ ]:
!python process_applyasync.py

In [ ]:
%%writefile process_mapasync.py
import multiprocessing
import time
 
def square(x):
    # This is is reeeeally slow way to square numbers.
    print(" excute process ",multiprocessing.current_process())
    time.sleep(0.5)
    return x**2
 
if __name__ == "__main__" : 
    print("Creating pool with 3 workers")
    pool = multiprocessing.Pool(processes=3)
    print("Invoking map_async(square, [11, 12, 13, 14, 15, 16])")
    result = pool.map_async(square, [11, 12, 13, 14, 15, 16])
    print( "Waiting for result...")
    start_time = time.time()
    print("Result: %r (%.2f secs)" % (result.get(), time.time() - start_time))
    pool.close()
    pool.join()

In [ ]:
!python process_mapasync.py

In [ ]:
%%writefile process_npsqrt.py

from multiprocessing import Pool, current_process
import numpy 

def sqrt(x):
    print(" process", current_process())
    return numpy.sqrt(x)

if __name__ == '__main__':
    pool = Pool(5)
    roots1 = pool.map(sqrt, range(6))
    print(roots1)
    roots2 = pool.map(sqrt, range(6,12))
    print(roots2)


In [ ]:
!python process_npsqrt.py

In [ ]:
%%writefile process_npsqrt1.py

from multiprocessing import Pool, current_process
import numpy

def sqrt(x):
    print(" process", current_process())
    return numpy.sqrt(x)


if __name__ == '__main__':
    
    pool = Pool()
    results = [pool.apply_async(sqrt, (x,)) for x in range(6)]
    print(results)
    roots = [r.get() for r in results]
    print(roots)

In [ ]:
!python process_npsqrt1.py

In [ ]:

import threading as th


# 호출자의 스레드 컨트롤 스레드 객체의 수
t = th.currentThread()
print(t)
print(th.current_thread())

# 활성 스레드 객체의 수
print(th.activeCount())
print(th.active_count())

#  현재 활성화 된 모든 스레드 객체의리스트
print(th.enumerate())
print(th.main_thread())



In [ ]:
%%writefile pool_map.py
from multiprocessing import Pool, current_process

def cube(x):
    print(current_process())
    return x**3

if __name__ == "__main__" :
    
    pool = Pool(processes=4)
    results = pool.map(cube, range(1,7))
    print(results)

In [ ]:
!python pool_map.py

In [ ]:
from multiprocessing import Pool
help(Pool().map_async)

In [ ]:
%%writefile pool_map1.py
from multiprocessing import Pool, current_process
import numpy as np

def worker(x):
    print(current_process())
    return np.add(x,x)

results_l = []
def result_back(result) :
    results_l.append(result)

if __name__ == "__main__" :
    
    pool = Pool(processes=4)
    # map
    result = pool.map_async(worker, [1, 2, 3], callback=result_back)
    
    print(result.get())
    print(results_l)
    pool.close()
    pool.join()

In [ ]:
!python pool_map1.py

In [ ]:
from threading import Lock
help(Lock().release)

In [ ]:
import threading 
import time

total = 0
def synchronized(func):
    func.__lock__ = threading.Lock()
    def synced_func(*args, **kws):
        with func.__lock__:
            return func(*args, **kws)
    return synced_func

@synchronized
def count():
    global total
    curr = total + 1
    time.sleep(0.1)
    print(threading.current_thread(), curr)
    total = curr
    

def counter():
    print(" exec thread", threading.current_thread())
    for i in range(0,10): 
        count()
        
thread1 = threading.Thread(target = counter)
thread2 = threading.Thread(target = counter)

thread1.start()
thread2.start()

thread1.join()
thread2.join()

print(total)

In [ ]:
import threading 
import time

total = 0
def counter():
    global total
        
    for i in range(0,10):
        print(threading.current_thread(),i)
        time.sleep(0.1)
        total += i
    
thread1 = threading.Thread(target = counter)
thread2 = threading.Thread(target = counter)

thread1.start()
thread2.start()

thread1.join()
thread2.join()

print(" total ", total)

In [ ]:
import threading

TOTAL = 0
MY_LOCK = threading.Lock()

class CountThread(threading.Thread):
    def run(self):
        global TOTAL
        for i in range(10):
            MY_LOCK.acquire()
            TOTAL = TOTAL + 1
            print("exex", threading.current_thread()._tstate_lock, TOTAL)
            MY_LOCK.release()
        print('%s\n' % (TOTAL))

a = CountThread()
b = CountThread()
a.start()
b.start()

In [ ]:
print(dir(threading.Lock()))

In [ ]:
dir(threading.Thread)

In [ ]:
help(threading.Lock().locked_lock)

In [ ]:
import threading
from queue import Queue
import time

q = Queue() 
# The worker thread pulls an item from the queue and processes it
def worker():
    print(" get thread ", threading.current_thread())
    while True:
        item = q.get()
        print(" get queue", item)
        q.task_done()
               
def queue_put() :
    print(" put thread ", threading.current_thread())
    for item in range(4):
        print(" put queue", item)
        q.put(item)

# Create the queue and thread pool.
threads = []
for i in range(2):
    t = threading.Thread(target=worker)
    threads.append(t)
    
for i in range(2):
    t = threading.Thread(target=queue_put)
    threads.append(t)
    
print(threads)
for t in threads :
    t.start()
    if q.full() == True :
        print(t,q.queue)

q.join()       # block until all tasks are done


In [ ]:
help(Queue.task_done)

In [ ]:
help(Queue.empty)
help(Queue.full)
help(Queue.join)
"""
 'empty',
 'full',
 'get',
 'get_nowait',
 'join',
 'put',
 'put_nowait',
 'qsize',
 'task_done'
 """

In [ ]:
from queue import Queue

q = Queue()
q.put("Hello")
print(q.queue)
q.put("world")
print(q.queue)

print("queue size ",q.qsize())
print( q.get() + ' ' + q.get())
print(q.queue)

print(q.empty())


In [ ]:
import threading

rl = threading.RLock()
print(rl)
print(rl.acquire())
print(rl.release())

In [ ]:
dir(threading.RLock())

In [ ]:
import threading
 
total = 0
lock = threading.RLock()
 
def do_something():
    with lock:
        print(lock)
    print('repease',lock) 
    return "Done doing something"
 
def do_something_else():
    with lock:
        print(lock)
    print('repease',lock) 
    return "Finished something else"
 
def main():
    with lock:
        result_one = do_something()
        result_two = do_something_else() 
    print (result_one)
    print (result_two)
    
main()

In [ ]:
import threading
 
total = 0
lock = threading.RLock()
 
def do_something():
    with lock:
        print(lock)
    print('repease',lock) 
    return "Done doing something"
 
def do_something_else():
    with lock:
        print(lock)
    print('repease',lock) 
    return "Finished something else"

def main():
    result_one = do_something()
    result_two = do_something_else() 
    print (result_one)
    print (result_two)

for i in range(2):
    my_thread = threading.Thread(target=main)
    my_thread.start()

In [ ]:
import threading
 
total = 0
lock = threading.RLock()
 
def update_total(amount):
    """
    Updates the total by the given amount
    """
    global total
    lock.acquire()
    try:
        total += amount
    finally:
        lock.release()
    print (threading.current_thread(),total)
    
if __name__ == '__main__':
    my_thread = threading.Thread(
            target=update_total, args=(5,))
    my_thread.start()
    

In [ ]:
import threading
 
total = 0
lock = threading.Lock()
 
def update_total(amount):
    """
    Updates the total by the given amount
    """
    global total
    with lock:
        total += amount
    print (threading.current_thread(),total)

    
if __name__ == '__main__':
    for i in range(10):
        my_thread = threading.Thread(
            target=update_total, args=(5,))
        my_thread.start()

In [3]:
import threading

sem = threading.BoundedSemaphore(3) 

print(sem)
print(sem._value)

print(sem.acquire())
print(sem._value)
print(sem.acquire())
print(sem._value)
print(sem.acquire())
print(sem._value)

print(sem.release())
print(sem._value)
print(sem.release())
print(sem._value)
print(sem.release())
print(sem._value)
print(sem.release())
print(sem._value)

3
True
2
True
1
True
0
None
1
None
2
None
3


ValueError: Semaphore released too many times

In [1]:
 
# 100개의 쓰레드 중에 3개의 쓰레드만 작업이 가능하다.

import threading

# 세마포 객체 생성, 3개의 쓰레드로 제한
sem = threading.Semaphore(3)      

class RestrictedArea(threading.Thread):
    def run(self):
        msg = 'Threading Semaphore TEST : %s' % self.getName()
        # 3개의 쓰레드만이 존재할수 있는 영역
        sem.acquire()
        print(sem._value,msg)                
        sem.release()              

threads = []

for i in range(10):
    threads.append(RestrictedArea())

for th in threads:
    th.start()          # 쓰레드 시작

for th in threads:
    th.join()           # 종료대기

print('Finish All Threading ')

2 Threading Semaphore TEST : Thread-6
2 Threading Semaphore TEST : Thread-7
2 Threading Semaphore TEST : Thread-8
2 Threading Semaphore TEST : Thread-9
2 Threading Semaphore TEST : Thread-10
2 Threading Semaphore TEST : Thread-11
2 Threading Semaphore TEST : Thread-12
2 Threading Semaphore TEST : Thread-13
2 Threading Semaphore TEST : Thread-14
2 Threading Semaphore TEST : Thread-15
Finish All Threading 
